# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [41]:
#!pip install selenium

In [42]:
# Lista de IMPORTAÇÃO quando for usar o SELENIUM
# chromedriver -> Chrome
# geckodriver -> Firefox

from selenium import webdriver # Criar o navegador
from selenium.webdriver.common.by import By # Localizar elementos( os itens de um site)
from selenium.webdriver.common.keys import Keys # Permite clicar tecla no teclado

navegador = webdriver.Chrome()

# Passo 1 - Entrar no google
navegador.get("https://www.google.com/")

# Passo 2 - Pesquisar a cotação do dólar
# Selecionando o Elemento do site

navegador.find_element(By.XPATH, 
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

navegador.find_element(By.XPATH, 
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]').send_keys(Keys.ENTER)

# Passo 3 - Pegar a cotação do dólar
cotacao_dolar = navegador.find_element(By.XPATH, 
                        '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print("Cotação do Dólar", cotacao_dolar)

# Passo 4 - Pegar a cotação do Euro
navegador.get("https://www.google.com/")

navegador.find_element(By.XPATH, 
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")

navegador.find_element(By.XPATH, 
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]').send_keys(Keys.ENTER)

# Passo 3 - Pegar a cotação do dólar
cotacao_euro = navegador.find_element(By.XPATH, 
                        '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print("Cotação do Euro ", cotacao_euro)

# Passo 5 - Pegar a cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')

# Formatação de numeros 
cotacao_ouro = cotacao_ouro.replace(",", ".")
print("Cotação do Ouro ", cotacao_ouro)


Cotação do Dólar 5.0598
Cotação do Euro  5.268213162
Cotação do Ouro  294.60


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [43]:
# Passo 6 - Atualiza a minha base de dados com as novas cotações
import pandas as pd

#Ler a base de Dados
tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [44]:
# Atualizar a cotação de acordo com a moeda correspondente

# Dólar
# Linhas onde a coluna moeda = dólar
# tabela.loc[linha, coluna] -> localizando dados na linha e coluna

tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)

# euro
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)

# ouro
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)


# Atualizar o preço de compra = preço  original  * cotação
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# Atualizar o preço de venda = preço de compra * margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.059800,5059.749402,1.40,7083.649163
1,Carro Renault,4500.00,Euro,5.268213,23706.959229,2.00,47413.918458
2,Notebook Dell,899.99,Dólar,5.059800,4553.769402,1.70,7741.407983
3,IPhone,799.00,Dólar,5.059800,4042.780200,1.70,6872.726340
4,Carro Fiat,3000.00,Euro,5.268213,15804.639486,1.90,30028.815023
5,Celular Xiaomi,480.48,Dólar,5.059800,2431.132704,2.00,4862.265408
6,Joia 20g,20.00,Ouro,294.600000,5892.000000,1.15,6775.800000


### Agora vamos exportar a nova base de preços atualizada

In [45]:
tabela.to_excel("TabelaAtualizada.xlsx", index=False)
navegador.quit()